<a href="https://colab.research.google.com/github/eminhan77/eminhan77/blob/main/Generative_Adversial_Network.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
!pip install keras
!pip install tensorflow
!pip install matplotlib
!pip install numpy
!pip install sklearn
!pip install pandas
!pip install h5py
!pip install pydot
!pip install graphviz
!pip install pydotplus


  error: subprocess-exited-with-error
  
  × python setup.py egg_info did not run successfully.
  │ exit code: 1
  ╰─> See above for output.
  
  note: This error originates from a subprocess, and is likely not a problem with pip.
  Preparing metadata (setup.py) ... error
error: metadata-generation-failed

× Encountered error while generating package metadata.
╰─> See above for output.

note: This is an issue with the package mentioned above, not pip.
hint: See above for details.


In [ ]:
import numpy as np
from matplotlib import pyplot
from keras.models import Sequential
from keras.layers import Dense
from keras.utils.vis_utils import plot_model

In [ ]:
def func_1D(x):
  return x*x

In [ ]:
inputs=np.arange(-0.5,0.6,0.1)
outputs=[func_1D(x) for x in inputs]

In [ ]:
pyplot.plot(inputs,outputs)
pyplot.show()

In [ ]:
#random values defined
def random_values(n=100):
  x1=np.random.rand(n)-0.5
  x2=x1*x1
  x1=x1.reshape(n,1)
  x2=x2.reshape(n,1)
  return np.hstack((x1,x2))

In [ ]:
#samples are generated and plotted
data=random_values()
pyplot.scatter(data[:,0],data[:,1])
pyplot.show()

In [ ]:
#discriminator UNIT
def disc(n_input=2):
  model=Sequential()
  model.add(Dense(25,activation='relu',kernel_initializer='he_uniform',input_dim=n_input))
  model.add(Dense(1,activation='sigmoid'))
  model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])
  return model


In [ ]:
#MODEL defined
model=disc()
model.summary()

In [ ]:
def real_SAMP(n):
  x1=np.random.rand(n)-0.5
  x2=x1*x1
  x1=x1.reshape(n,1)
  x2=x2.reshape(n,1)
  X=np.hstack((x1,x2))
  y=np.ones((n,1))
  return X,y

In [ ]:
def FAKE_samp(n):
  x1=-1+np.random.rand(n)*2
  x2=-1+np.random.rand(n)*2
  x1=x1.reshape(n,1)
  x2=x2.reshape(n,1)
  X=np.hstack((x1,x2))
  y=np.zeros((n,1))
  return X,y

In [ ]:
#train thE MODEL
def trainn_MODEL(model,n_epochs=1000,n_batch=128):
  half_BATCH=int(n_batch/2)
  for i in range(n_epochs):
    X_real,y_real=real_SAMP(half_BATCH)
    X_fake,y_fake=FAKE_samp(half_BATCH)
    model.train_on_batch(X_real,y_real)
    model.train_on_batch(X_fake,y_fake)
    _,acc_real=model.evaluate(X_real,y_real,verbose=0)
    _,acc_fake=model.evaluate(X_fake,y_fake,verbose=0)
  return model

In [ ]:
model=disc()
trainn_MODEL(model)

In [7]:
#THe GENERATOR model comes into play

def gen(latent_dim,n_outputs=2):
  model=Sequential()
  model.add(Dense(15,activation='relu',kernel_initializer='he_uniform',input_dim=latent_dim))
  model.add(Dense(n_outputs,activation='linear'))
  return model

SyntaxError: incomplete input (<ipython-input-7-353a455d4501>, line 3)

In [ ]:
model=gen(5)
model.summary()

In [ ]:
#POINTS in latent SPACE are generated

def gen_points(latent_dim,n_samples):
  x_input=np.random.rand(latent_dim*n_samples)
  x_input=x_input.reshape(n_samples,latent_dim)
  return x_input


In [ ]:
def fake_samples(g_model,latent_dim,n_samples):
  x_input=gen_points(latent_dim,n_samples)
  X=g_model.predict(x_input)
  y=np.zeros((n_samples,1))
  return X,y

In [ ]:
def gan(generator,discriminator):
  discriminator.trainable=False
  model=Sequential()
  model.add(generator)
  model.add(discriminator)
  model.compile(loss='binary_crossentropy',optimizer='adam')
  return model


In [ ]:
def train(g_model,d_model,gan_model,latent_dim,n_epochs=100,n_batch=128,n_eval=2000):
  half_batch=int(n_batch/2)
  for i in range(n_epochs):
    x_real,y_real=real_SAMP(half_batch)
    x_fake,y_fake=fake_samples(g_model,latent_dim,half_batch)
    d_model.train_on_batch(x_real,y_real)
    d_model.train_on_batch(x_fake,y_fake)
    x_gan=gen_points(latent_dim,n_batch)
    y_gan=np.ones((n_batch,1))
    gan_model.train_on_batch(x_gan,y_gan)
    if((i+1)%n_eval==0):
      summorize_performance(i,g_model,d_model,latent_dim)

In [ ]:
def perforMANCE(epoch,g_model,d_model,latent_dim,n_samples=100):
  x_real,y_real=real_SAMP(n_samples)
  _,acc_real=d_model.evaluate(x_real,y_real,verbose=0)
  x_fake,y_fake=fake_samples(g_model,latent_dim,n_samples)
  _,acc_fake=d_model.evaluate(x_fake,y_fake,verbose=0)
  print(epoch,acc_real,acc_fake)

  pyplot.scatter(x_real[:,0],x_real[:,1],color='red')
  pyplot.scatter(x_fake[:,0],x_fake[:,1],color='blue')
  pyplot.close()

In [ ]:
latent_dim=5
d_model=disc()
g_model=gen(latent_dim)
gan_model=gan(g_model,d_model)
train(g_model,d_model,gan_model,latent_dim)